# 第09章 — 遺伝的アルゴリズム

- 候補解の集団を選択・交叉・突然変異で進化させる。
- 適応度が選択圧を決め、多様性が早すぎる収束を防ぐ。
- 表現（文字列、ベクトル、木など）に合わせた演算が重要。
- 確率的選択（ルーレット、トーナメント）が探索と活用のバランスを取る。

試してみよう: POPを増やしたりMUTATIONを下げて収束速度と多様性の変化を見る。

In [ ]:
# 文字列GAの設定とオペレータを定義
import random

TARGET = "nature"
POP = 20
MUTATION = 0.05

def random_gene():
    return chr(random.randint(97, 122))

def fitness(chrom):
    return sum(c1 == c2 for c1, c2 in zip(chrom, TARGET))

def crossover(a, b):
    point = random.randrange(len(TARGET))
    return a[:point] + b[point:]


In [ ]:
# 集団を進化させてターゲット文字列に近づける
population = [''.join(random_gene() for _ in TARGET) for _ in range(POP)]
generation = 0

while True:
    scored = sorted(((fitness(c), c) for c in population), reverse=True)
    best_score, best = scored[0]
    print(f"gen {generation}: best='{best}' score={best_score}")
    if best_score == len(TARGET) or generation >= 15:
        break

    mating_pool = [c for score, c in scored for _ in range(score + 1)]
    new_pop = []
    for _ in range(POP):
        parent1 = random.choice(mating_pool)
        parent2 = random.choice(mating_pool)
        child = ''.join(
            (crossover(parent1, parent2)[i] if random.random() > MUTATION else random_gene())
            for i in range(len(TARGET))
        )
        new_pop.append(child)

    population = new_pop
    generation += 1


### 追加例: 連続値GAで関数最大化

In [ ]:
# 連続値GAの評価関数とパラメータ
import math
import random

def fitness(x):
    return math.sin(5 * x) * x + math.cos(3 * x)

POP = 20
MUT = 0.1
BOUNDS = (0.0, 2.0)
pop = [random.uniform(*BOUNDS) for _ in range(POP)]

def select(pop):
    scores = [(fitness(x), x) for x in pop]
    scores.sort(reverse=True)
    pool = [x for score, x in scores for _ in range(2)]
    return scores[0], pool

def crossover(a, b):
    return (a + b) / 2


In [ ]:
# 連続値GAを複数世代回してスコア確認
generation = 0
while generation < 12:
    (best_score, best_x), pool = select(pop)
    print(f"gen {generation}: best x={best_x:.3f} f={best_score:.3f}")
    if generation == 11:
        break
    new_pop = []
    for _ in range(POP):
        pa, pb = random.choice(pool), random.choice(pool)
        child = crossover(pa, pb)
        if random.random() < MUT:
            child += random.uniform(-0.1, 0.1)
        child = min(max(child, BOUNDS[0]), BOUNDS[1])
        new_pop.append(child)
    pop = new_pop
    generation += 1
